In [0]:
#!unzip zomato.zip

In [0]:
import numpy as np
import pandas as pd
import nltk

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [269]:
df = pd.read_csv('zomato/processed_reviews.csv')
print(type(df))
df.head(5)

<class 'pandas.core.frame.DataFrame'>


,user_id,entity_id,review_id,datetime,rating,content
0,2042820,44568,44166735,2019-07-25 12:19:02,1.0,i have been a very loyal customer to global fu...
1,57816837,44568,44134010,2019-07-23 00:36:53,5.0,visited first time on my visit to mumbai and f...
2,42096287,44568,44132846,2019-07-22 23:10:33,5.0,this place is an absolute delight of you're a ...
3,33538931,44568,44082978,2019-07-20 04:42:05,5.0,this is one of the coolest places for a buffet...
4,34390186,44568,44021065,2019-07-15 22:29:08,4.0,"good food, has vast variety to choose from, su..."


In [270]:
df.shape

(3228, 6)

In [0]:
zomato_data = df[['entity_id', 'user_id' , 'rating', 'content']]

In [272]:
zomato_data.head()

,entity_id,user_id,rating,content
0,44568,2042820,1.0,i have been a very loyal customer to global fu...
1,44568,57816837,5.0,visited first time on my visit to mumbai and f...
2,44568,42096287,5.0,this place is an absolute delight of you're a ...
3,44568,33538931,5.0,this is one of the coolest places for a buffet...
4,44568,34390186,4.0,"good food, has vast variety to choose from, su..."


In [0]:
def int2str(num):
  return str(num)

zomato_data['user_id'] = zomato_data['user_id'].apply(int2str)
zomato_data['entity_id'] = zomato_data['entity_id'].apply(int2str)

In [274]:
print(type(zomato_data['entity_id'][0]))
zomato_data.head()

<class 'str'>


,entity_id,user_id,rating,content
0,44568,2042820,1.0,i have been a very loyal customer to global fu...
1,44568,57816837,5.0,visited first time on my visit to mumbai and f...
2,44568,42096287,5.0,this place is an absolute delight of you're a ...
3,44568,33538931,5.0,this is one of the coolest places for a buffet...
4,44568,34390186,4.0,"good food, has vast variety to choose from, su..."


### Text Cleaning

In [275]:
#remove emoji and remove special characters
!pip install emoji
import emoji
bad_chars = [';', ':', '!', "*",'.',','] 
def remove_special_characters(text):
  # text = give_emoji_free_text(text);
  return filter(lambda i: i not in bad_chars, text);
# def give_emoji_free_text(text):
#     return emoji.get_emoji_regexp().sub(r'', text.encode().decode('utf8'))

In [0]:
#zomato_data['user_id'] = zomato_data['user_id'].astype(str)

In [277]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords
import string
stop = []
for word in stopwords.words('english'):
  s = [char for char in word if char not in string.punctuation]
  stop.append(''.join(s))


In [0]:
def text_process(text) :
  text = remove_special_characters(text)
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)
  return " ".join([word for word in nopunc.split() if word.lower() not in stop])

In [0]:
zomato_data['content'] = zomato_data['content'].apply(text_process)

In [281]:
zomato_data['content'][0]

'loyal customer global fusion past years past times global fusion disapointing firstly went fruits salads wich r alwys der lik mango papaya secondly fork given wat ws big size felt like serving fork eating one nobody came ask drinks needed tepenyaki prawns absolute disappointment wer jus plain fried taste momos wer extemely dry iceberg iceberg chkn crispy corn pepper mushroom salty particular name dishes bcz hv regular versova n andheri branch hv biggest fan bt smhow bandra worli dsnt appeal dt mch mean global fusion known spread salad go wd appetisers cherry cake wen v went hv desserts askd fr tea spoon said dey u u belive small gulab jamun n icecream big serving spoons mean hw cn u nt hv dessert spoon restaurant lik global fusion u reli need work cz u may lose loyal client'

### Stemming

### Train, Test, Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
vld_size = 0.15
X_train, X_valid, y_train, y_valid = train_test_split(zomato_data['content'], df['entity_id'], test_size = vld_size)

In [0]:
userid_df = zomato_data[['user_id', 'content']]
businessid_df = zomato_data[['entity_id', 'content']]

In [285]:
userid_df.head()

,user_id,content
0,2042820,loyal customer global fusion past years past t...
1,57816837,visited first time visit mumbai fell love vari...
2,42096287,place absolute delight foodie spread vast many...
3,33538931,one coolest places buffet lunch dinner asian c...
4,34390186,good food vast variety choose sushi wasabi one...


In [0]:
userid_df = userid_df.groupby('user_id').agg({'content': ' '.join})
businessid_df = businessid_df.groupby('entity_id').agg({'content': ' '.join})

In [0]:
# result_df = userid_df.loc[userid_df['user_id'] == '34390186']['content']
# print(result_df)

In [288]:
userid_df.head(10)

,content
user_id,
100083557,diwali corporate party people place massive sp...
100084362,good place enjoy family friends suitable birth...
100190388,randomly plan awesome even random choice food ...
10026121,good food
1004368,walked overseas clients buffet table available...
100504193,quality concern fatty patty owner polite helpf...
100505080,school mates meet house long time order food s...
100656812,authentic punjabi chole bhature samosa jalebi ...
100937,suggest booking go big restaurant tables quite...


In [289]:
userid_df['content']

user_id
100083557    diwali corporate party people place massive sp...
100084362    good place enjoy family friends suitable birth...
100190388    randomly plan awesome even random choice food ...
10026121                                             good food
1004368      walked overseas clients buffet table available...
                                   ...                        
9946621      fatty patty mouthwatering burgers super reason...
9961881      arranged birthday celebration food really awes...
99631839     one best place town visit really amazing exper...
99718960     great experience quality taste amazing best pl...
997230       crowd really good try place loved surely gonna...
Name: content, Length: 2928, dtype: object

In [290]:
businessid_df.head()

,content
entity_id,
18126406,hotel good food wow ambience amazing staff goo...
18234208,want try real delhi food place ive best malai ...
18306468,go big groups let order platters zomato gold a...
18374844,cafe located ground floor office building bogo...
18555111,like taste paneer fresh zomato sucks schezwan ...


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer

In [292]:
userid_vectorizer = TfidfVectorizer(tokenizer = RegexpTokenizer('[a-zA-Z]+').tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['content'])
userid_vectors.shape

(2928, 5000)

In [293]:
userid_vectors

<2928x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 109941 stored elements in Compressed Sparse Row format>

In [294]:
businessid_vectorizer = TfidfVectorizer(tokenizer = RegexpTokenizer('[a-zA-Z]+').tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(businessid_df['content'])
businessid_vectors.shape

(37, 5000)

## Matrix Factorisation

In [295]:
userid_rating_matrix = pd.pivot_table(zomato_data, values= 'rating', index=['user_id'], columns=['entity_id'])
userid_rating_matrix.shape

(2928, 37)

In [296]:
userid_rating_matrix.head(10)

entity_id,18126406,18234208,18306468,18374844,18555111,18582428,18586520,18635066,18635358,18643193,18662496,18663481,18721192,18727155,18784164,18968310,18973846,18983744,18995602,19018039,19037741,19093447,19122846,19164221,19261600,19277557,34212,34336,34362,36867,37823,38410,38495,43911,44568,48323,48688
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100083557,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100084362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
100190388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
10026121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
1004368,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100504193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100505080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100656812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN


In [0]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=businessid_df.index, columns=businessid_vectorizer.get_feature_names())

In [298]:
print(P.shape)
print(Q.shape)
print(userid_rating_matrix.shape)

(2928, 5000)
(37, 5000)
(2928, 37)


In [299]:
P.head()
#print(P.shape)

,aa,aam,aanch,abhishek,able,absolut,absolute,absolutely,abundant,ac,accept,acceptable,accepted,access,accessible,accommodate,accommodating,accommodation,accomodating,accompanied,accompaniment,according,accordingly,achari,across,acted,action,active,actual,actually,ad,adana,add,added,adding,addition,additional,address,adds,adequate,...,ye,yeah,year,yearold,years,yellow,yes,yest,yesterday,yet,yoghurt,yogurt,yoktori,yolk,young,younger,youngsters,youth,yrs,yum,yumm,yummiest,yummilicious,yummm,yummmm,yummmy,yummy,yuzu,z,zafrani,zam,zauqeshahi,zero,zest,zesty,zillion,zomato,zomatos,zone,zucchini
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100083557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100084362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100190388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10026121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.203583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [300]:
Q.head()

,aa,aam,aanch,abhishek,able,absolut,absolute,absolutely,abundant,ac,accept,acceptable,accepted,access,accessible,accommodate,accommodating,accommodation,accomodating,accompanied,accompaniment,according,accordingly,achari,across,acted,action,active,actual,actually,ad,adana,add,added,adding,addition,additional,address,adds,adequate,...,ye,yeah,year,yearold,years,yellow,yes,yest,yesterday,yet,yoghurt,yogurt,yoktori,yolk,young,younger,youngsters,youth,yrs,yum,yumm,yummiest,yummilicious,yummm,yummmm,yummmy,yummy,yuzu,z,zafrani,zam,zauqeshahi,zero,zest,zesty,zillion,zomato,zomatos,zone,zucchini
entity_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18126406,0.0,0.0,0.0,0.004316,0.002954,0.000000,0.000000,0.011353,0.0,0.003646,0.003646,0.011509,0.000000,0.008112,0.003328,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.002658,0.004056,0.004634,0.002954,0.004634,0.0,0.000000,0.000000,0.005143,0.0,0.0,0.002658,0.000000,0.00000,0.0,0.000000,0.004056,0.000000,0.008112,...,0.000000,0.0,0.002954,0.0,0.024751,0.008632,0.012066,0.0,0.010287,0.005143,0.0,0.004056,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.004316,0.017090,0.006656,0.0,0.0,0.000000,0.005044,0.0,0.013386,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.004160,0.0,0.000000,0.0
18234208,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.012397,0.018348,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.005763,0.002341,0.000000,0.0,0.0,0.005763,0.0,0.000000,0.000000,0.005350,0.003410,0.000000,0.0,0.007474,0.004982,0.000000,0.0,0.0,0.004603,0.010231,0.00535,0.0,0.000000,0.000000,0.003542,0.000000,...,0.011646,0.0,0.001705,0.0,0.001587,0.004982,0.006965,0.0,0.007422,0.005938,0.0,0.000000,0.0,0.0,0.000000,0.002911,0.000000,0.0,0.000000,0.006577,0.007684,0.0,0.0,0.002491,0.000000,0.0,0.019869,0.0,0.0,0.0,0.0,0.0,0.000000,0.002911,0.0,0.0,0.002401,0.0,0.000000,0.0
18306468,0.0,0.0,0.0,0.000000,0.004257,0.007269,0.000000,0.006544,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011057,0.0,0.0,0.004796,0.0,0.000000,0.000000,0.000000,0.000000,0.006678,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.007661,0.000000,0.00000,0.0,0.005845,0.005845,0.000000,0.000000,...,0.000000,0.0,0.004257,0.0,0.003963,0.000000,0.006955,0.0,0.003706,0.000000,0.0,0.000000,0.0,0.0,0.005845,0.000000,0.013356,0.0,0.006220,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.005512,0.0,0.0,0.0,0.0,0.0,0.006678,0.000000,0.0,0.0,0.032976,0.0,0.013356,0.0
18374844,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
18555111,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.007345,0.0,0.000000,0.000000,0.000000,0.011795,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.008599,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.008599,0.000000,0.00000,0.0,0.000000,0.013121,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.023420,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.018559,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.067294,0.0,0.000000,0.0


In [0]:
# print(Q.shape)
# print(userid_rating_matrix.index)
# print(userid_rating_matrix)

### Gradient Descent optimization

In [0]:
def matrix_factorization(R, P, Q, steps=5, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j] > 0:
                    # print(P.loc[i])
                    #print(Q.loc[j])
                    #print(j)
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    #print("error 1")
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    #print("error 2")
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
                    #print("error 3")
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j] > 0:
                    e = e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
    return P,Q

In [303]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

CPU times: user 8min 54s, sys: 121 ms, total: 8min 54s
Wall time: 8min 55s


In [304]:
Q.head()

,aa,aam,aanch,abhishek,able,absolut,absolute,absolutely,abundant,ac,accept,acceptable,accepted,access,accessible,accommodate,accommodating,accommodation,accomodating,accompanied,accompaniment,according,accordingly,achari,across,acted,action,active,actual,actually,ad,adana,add,added,adding,addition,additional,address,adds,adequate,...,ye,yeah,year,yearold,years,yellow,yes,yest,yesterday,yet,yoghurt,yogurt,yoktori,yolk,young,younger,youngsters,youth,yrs,yum,yumm,yummiest,yummilicious,yummm,yummmm,yummmy,yummy,yuzu,z,zafrani,zam,zauqeshahi,zero,zest,zesty,zillion,zomato,zomatos,zone,zucchini
entity_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18126406,1.356443e-04,9.094747e-06,2.165717e-04,0.025742,0.014875,2.572333e-05,0.004039,0.065782,1.099262e-04,0.047192,0.010188,4.727302e-02,9.793526e-04,0.033767,0.024910,0.000443,0.000359,4.145232e-04,2.617641e-04,0.000280,0.000142,0.040940,0.012219,0.016065,0.019120,0.013353,3.188087e-04,3.912015e-04,0.000227,0.029082,2.264538e-04,0.000058,0.022007,0.004784,2.556263e-04,0.000199,0.000124,1.207024e-02,0.011936,0.052434,...,2.465120e-04,2.368067e-05,0.014526,0.000079,0.173517,2.892177e-02,0.049416,2.123351e-05,0.061948,0.049671,0.000118,0.013504,2.748713e-06,2.532662e-05,6.011239e-05,3.268097e-04,8.705651e-05,2.476748e-05,2.955503e-02,0.088634,0.037422,6.090007e-05,8.763038e-05,2.274688e-04,2.595302e-02,0.000106,0.129583,6.306436e-06,7.924382e-06,0.000148,1.553711e-04,0.008962,8.231275e-05,7.887388e-05,3.404040e-06,1.826534e-02,0.074351,8.195836e-05,9.139856e-05,0.000054
18234208,4.096143e-05,7.764037e-05,5.786553e-05,0.000473,0.000496,1.079642e-04,0.038857,0.142164,1.549841e-04,0.001421,0.004842,4.651012e-04,5.254959e-05,0.009504,0.025122,0.014317,0.000153,6.267950e-05,1.460776e-04,0.030963,0.000148,0.002668,0.000512,0.019469,0.021294,0.000140,3.139794e-04,2.652302e-02,0.024065,0.001256,1.388424e-04,0.000608,0.028444,0.050642,2.637577e-02,0.000256,0.000127,1.012076e-04,0.013150,0.000353,...,3.720876e-02,2.203597e-04,0.016521,0.000129,0.011304,2.011134e-02,0.043700,2.589197e-04,0.044026,0.042842,0.011241,0.001040,1.719441e-04,3.150704e-05,5.071006e-04,1.482492e-02,3.064958e-04,3.482959e-04,1.755293e-04,0.055385,0.036469,1.798858e-04,1.211343e-04,1.430949e-02,1.162987e-04,0.000089,0.118464,5.847519e-04,2.902701e-04,0.000359,6.884673e-05,0.000169,3.123927e-05,1.874652e-02,1.338746e-04,1.847509e-04,0.019326,9.265433e-06,8.689845e-05,0.000376
18306468,1.480481e-04,1.153346e-05,2.063713e-05,0.000160,0.015381,5.313127e-02,0.006485,0.064385,1.297435e-04,0.000509,0.000426,3.227561e-04,2.961039e-05,0.022111,0.000411,0.000212,0.076192,3.563487e-05,3.315538e-05,0.014207,0.000088,0.000651,0.000366,0.000043,0.006341,0.013579,1.554584e-04,2.761655e-04,0.000045,0.000566,1.077635e-04,0.000110,0.021411,0.000816,1.269591e-04,0.000182,0.029035,1.230641e-02,0.000606,0.000038,...,1.466267e-04,3.436000e-05,0.045527,0.000003,0.008047,1.075193e-04,0.028938,3.938748e-05,0.028846,0.000571,0.000098,0.000225,4.508900e-05,2.145828e-07,3.301459e-02,2.389289e-05,8.491105e-02,5.768760e-05,2.595663e-02,0.000875,0.000379,3.493632e-05,3.862576e-05,5.479824e-05,2.099877e-05,0.000121,0.055543,1.578720e-04,8.275782e-05,0.000158,1.569451e-04,0.000006,1.085616e-02,9.058312e-05,3.321982e-05,1.588850e-04,0.093823,6.276684e-05,6.563739e-02,0.000064
18374844,4.241470e-08,1.497886e-08,6.797244e-08,0.000013,0.000019,3.209629e-08,0.000026,0.000066,6.668692e-08,0.000014,0.000008,1.582169e-07,9.715755e-08,0.000009,0.000011,0.000063,0.000007,9.696865e-08,9.758028e-08,0.000008,0.000010,0.000008,0.000029,0.000065,0.000020,0.000008,1.068548e-07,1.626169e-07,0.000019,0.000063,1.024371e-07,0.000002,0.000018,0.000022,1.471146e-07,0.000002,0.000002,1.139391e-07,0.000035,0.000014,...,1.202366e-07,1.044473e-07,0.000010,0.000020,0.000015,1.370069e-07,0.000037,5.989805e-08,0.000045,0.000069,0.000022,0.000085,5.422933e-08,5.553535e-06,1.126334e-07,1.314138e-07,8.201716e-08,7.785086e-08,8.686266e-08,0.000038,0.000

In [305]:
Q.iloc[0].sort_values(ascending=False).head(10)

good        1.589176
food        1.578324
buffet      1.418196
place       1.356206
ambience    0.923083
service     0.857525
family      0.771796
starters    0.717193
great       0.688942
sai         0.658012
Name: 18126406, dtype: float64

### Export Model

In [0]:

# Store P, Q and vectorizer in pickle file
import pickle
output = open('yelp_recommendation_model.pkl', 'wb')
pickle.dump(P,output)
pickle.dump(Q,output)
pickle.dump(userid_vectorizer,output)
output.close()

In [0]:
df_business = pd.read_csv('zomato/restaurant.csv')

In [0]:
df_business['entity_id'] = df_business['entity_id'].apply(int2str)

In [0]:
#result = df_business.loc[df_business['entity_id'] == 18983744]['name'].iloc[0]

In [310]:
result

'Stoned Over Shakes-Hashtag By H&A'

In [312]:
words = "best barbeque"
test_df= pd.DataFrame([words], columns=['content'])
test_df['content'] = test_df['content'].apply(text_process)
test_vectors = userid_vectorizer.transform(test_df['content'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['rating'],ascending=[0])[:7]
for i in topRecommendations.index:
    print("Name : " + df_business[df_business['entity_id']==i]['name'].iloc[0])
    print("Cuisines : " + df_business[df_business['entity_id']==i]['cuisines'].iloc[0])
    print("People liked : " + str(df_business[df_business['entity_id'] == i]['people_liked'].iloc[0]))
    print("Ratings : " + str(df_business[df_business['entity_id']==i]['avg_review'].iloc[0])+ "\nReviews : "+str(df_business[df_business['entity_id']==i]['review_count'].iloc[0]))
    print('\n')

Name : Barbeque Nation
Cuisines : North Indian,Kebab,BBQ,Chinese
People liked : Kulfi,Fish,Gulab Jamun,Biryani,Crispy Corn,Phirni,Chicken Drumstick,Decor,Live Grill,Comfortable Seating,Courteous Staff,Friendly Staff,Friendly Service
Ratings : 4.2
Reviews : 2902


Name : Gulati's
Cuisines : Mithai,Street Food
People liked : Chole Bhature,Samosa,Motichoor Ladoo,Rasmalai,Jalebi Rabri,Chana Bhatura,Kachori,Prompt Service
Ratings : 4.2
Reviews : 1382


Name : Global Fusion
Cuisines : Chinese,Japanese,Asian,North Indian
People liked : Sushi,Sea Food,Dumplings,Momos,Mutton Curry,Tempura Prawns,Chicken Bao,Decor,Lovely Ambience,Elegant Ambience,Courteous Staff,Friendly Staff,Friendly Service
Ratings : 4.2
Reviews : 5601


Name : Banjara - Goldfinch Hotel
Cuisines : North Indian,Continental,Chinese
People liked : Mocktails,Fish,Panipuri,Salad,Biryani,Momos,Lunch Buffet,Elegant Decor,Lovely Ambience,Rooftop Ambience,Courteous Staff,Friendly Staff
Ratings : 4.5
Reviews : 674


Name : Mitron - Pen